In [1]:
import numpy as np
import os

import albumentations as A
from albumentations.pytorch import ToTensorV2
import wandb
from dotenv import load_dotenv

from src.model.model import save_model, load_model
from src.dataset.df import df_dataset, check_dataset
from src.utils.common import set_seed
from src.utils.optim_thresh import calc_optim_thresh
from src.experiment.experiment import train, valid
from src.experiment.initialize import init_dataset, init_model, init_exp

In [2]:
class cfg:
    debug = False
    check_dataset = False

    # = data CFG ====================================================

    dataset_path = "/kaggle/working/dataset/cropped_xy_256_128_z_6_6/"
    dataset = "base2d"

    # = experiment CFG =================================================

    project = "SenNet"
    exp_name = os.path.basename(os.getcwd())
    notes = "aug_RandomRotate90"

    # = model CFG ======================================================

    model_arch = "Unet"
    backbone = "efficientnet-b0"
    in_chans = 6
    target_size = 6

    # = training CFG ===================================================

    epochs = 20

    train_batch_size = 128
    valid_batch_size = train_batch_size

    loss = "DiceLoss"
    metrics = "Dice"
    lr = 1e-4
    thresholds_to_test = range(2, 101, 4)
    num_workers = 24

    # = augmentation ===================================================

    image_size = 256
    train_aug = [
        # A.RandomResizedCrop(image_size, image_size, scale=(0.8, 1.0)),
        A.RandomRotate90(p=0.75),
        # A.ShiftScaleRotate(p=0.75),
        # A.OneOf(
        #     [
        #         A.GaussNoise(var_limit=[10, 50]),
        #         A.GaussianBlur(),
        #         A.MotionBlur(),
        #     ],
        #     p=0.4,
        # ),
        # A.GridDistortion(num_steps=5, distort_limit=0.3, p=0.5),
        ToTensorV2(transpose_mask=True),
    ]

    valid_aug = [
        ToTensorV2(transpose_mask=True),
    ]


load_dotenv("/kaggle/key.env")
set_seed()

In [3]:
def filter_dataset(df):
    # labelが全くないものは90%の確率で除外
    df["random"] = np.random.rand(len(df))
    df = df[(df["sum"] > 0) | (df["random"] < 0.1)]
    df = df.reset_index(drop=True)
    df = df.drop(["random"], axis=1)
    return df


df = df_dataset(cfg)
df = filter_dataset(df)
# df = df.sample(1000).reset_index(drop=True)
if cfg.debug:
    df = df.sample(10000).reset_index(drop=True)
display(df)

if cfg.check_dataset:
    check_dataset(df, cfg)

,image_path,label_path,fname,kidney,x,y,z,std,sum,fold0,fold1
0,/kaggle/working/dataset/cropped_xy_256_128_z_6...,/kaggle/working/dataset/cropped_xy_256_128_z_6...,x0_y0_z12_std0040_sum0,kidney_1_dense,0,0,12,40,0,train,valid
1,/kaggle/working/dataset/cropped_xy_256_128_z_6...,/kaggle/working/dataset/cropped_xy_256_128_z_6...,x0_y0_z150_std0033_sum0,kidney_1_dense,0,0,150,33,0,train,valid
2,/kaggle/working/dataset/cropped_xy_256_128_z_6...,/kaggle/working/dataset/cropped_xy_256_128_z_6...,x0_y0_z252_std0036_sum0,kidney_1_dense,0,0,252,36,0,train,valid
3,/kaggle/working/dataset/cropped_xy_256_128_z_6...,/kaggle/working/dataset/cropped_xy_256_128_z_6...,x0_y0_z270_std0036_sum0,kidney_1_dense,0,0,270,36,0,train,valid
4,/kaggle/working/dataset/cropped_xy_256_128_z_6...,/kaggle/working/dataset/cropped_xy_256_128_z_6...,x0_y0_z300_std0037_sum0,kidney_1_dense,0,0,300,37,0,train,valid
...,...,...,...,...,...,...,...,...,...,...,...
37926,/kaggle/working/dataset/cropped_xy_256_128_z_6...,/kaggle/working/dataset/cropped_xy_256_128_z_6...,x256_y896_z972_std0228_sum3897,kidney_3_sparse,256,896,972,228,3897,,train
37927,/kaggle/working/dataset/cropped_xy_256_128_z_6...,/kaggle/working/dataset/cropped_xy_256_128_z_6...,x256_y896_z978_std0223_sum5084,kidney_3_sparse,256,896,978,223,5084,,train
37928,/kaggle/working/dataset/cropped_xy_256_128_z_6...,/kaggle/working/dataset/cropped_xy_256_128_z_6...,x256_y896_z984_std0223_sum6730,kidney_3_sparse,256,896,984,223,6730,,train
37929,/kaggle/working/dataset/cropped_xy_256_128_z_6...,/kaggle/working/dataset/cropped_xy_256_128_z_6...,x256_y896_z990_std0227_sum7998,kidney_3_sparse,256,896,990,227,7998,,train


In [4]:
if cfg.debug:
    print("!!!Debug mode!!!\n")
    cfg.epochs = 5

for fold in range(2):
    train_dataloader, valid_dataloader = init_dataset(fold, df, cfg)
    model, scaler, criterion, optimizer, scheduler, metrics = init_model(cfg)
    slacknotify = init_exp(fold, cfg)

    path_best = f"./{cfg.exp_name}/{cfg.exp_name}_best_fold{fold}.pth"
    path_last = f"./{cfg.exp_name}/{cfg.exp_name}_last_fold{fold}.pth"

    best_loss = float("inf")
    for epoch in range(cfg.epochs):
        train(model, train_dataloader, optimizer, criterion, scheduler, scaler, epoch, cfg)
        loss, pred_list, true_list = valid(model, valid_dataloader, criterion, epoch, cfg)

        if loss < best_loss:
            print(f"loss : {loss:.4f}\tSAVED MODEL\n")
            slacknotify.send_reply(f"epoch : {epoch}\tscore : {loss:.4f}\tBEST")
            best_loss = loss
            save_model(model, cfg, path_best, loss=loss)
        else:
            print(f"loss : {loss:.4f}\n")
            slacknotify.send_reply(f"epoch : {epoch}\tscore : {loss:.4f}")

    last_score, last_thresh = calc_optim_thresh(pred_list, true_list, metrics, cfg)
    save_model(model, cfg, path_last, loss=loss, score=last_score, thresh=last_thresh)
    wandb.config.update({"last_score": last_score, "last_thresh": last_thresh})

    best_model = load_model(model, path_best)
    loss, pred_list, true_list = valid(best_model, valid_dataloader, criterion, epoch, cfg, log=False)

    best_score, best_thresh = calc_optim_thresh(pred_list, true_list, metrics, cfg)
    save_model(best_model, cfg, path_best, loss=loss, score=best_score, thresh=best_thresh)
    wandb.config.update({"best_score": best_score, "best_thresh": best_thresh})

    slacknotify.send_reply(
        f"{cfg.exp_name}_fold{fold} training finished\nbest score : {best_score:.4f} last score : {last_score:.4f}",
        True,
    )

    if wandb.run:
        wandb.finish()

model_arch:  Unet
backbone:  efficientnet-b0


wandb: Currently logged in as: welshonionman. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /kaggle/wandb/run-20240101_233216-srhv3qgk
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run exp007_fold0
wandb: ⭐️ View project at https://wandb.ai/welshonionman/SenNet
wandb: 🚀 View run at https://wandb.ai/welshonionman/SenNet/runs/srhv3qgk
Epoch 0/20  Mem : 14.7GB  LR : 1.00E-04  Loss: 0.9272: 100%|██████████| 237/237 [01:49<00:00,  2.16it/s]
Val Loss: 0.9473: 100%|██████████| 30/30 [00:18<00:00,  1.60it/s]                                                                                                                                                                                                                                      


loss : 0.9473	SAVED MODEL



Epoch 1/20  Mem : 19GB  LR : 1.00E-03  Loss: 0.2136: 100%|██████████| 237/237 [01:41<00:00,  2.34it/s]
Val Loss: 0.4210: 100%|██████████| 30/30 [00:18<00:00,  1.61it/s]                                                                                                                                                                                                                                      


loss : 0.4210	SAVED MODEL



...

Epoch 18/20  Mem : 19GB  LR : 9.64E-06  Loss: 0.0593: 100%|██████████| 189/189 [01:23<00:00,  2.25it/s]
Val Loss: 0.4389: 100%|██████████| 109/109 [00:57<00:00,  1.88it/s]                                                                                                                                                                                                                                    


loss : 0.4389



Epoch 19/20  Mem : 19GB  LR : 5.54E-06  Loss: 0.0588: 100%|██████████| 189/189 [01:23<00:00,  2.27it/s]
Val Loss: 0.4341: 100%|██████████| 109/109 [00:58<00:00,  1.87it/s]                                                                                                                                                                                                                                    


loss : 0.4341



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [08:16<00:00, 19.86s/it]
wandb:                                                                                
wandb: 
wandb: Run history:
wandb:      epoch ▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
wandb: train_loss █▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: valid_loss ██▅▂▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁
wandb: 
wandb: Run summary:
wandb:      epoch 19
wandb: train_loss 0.07811
wandb: valid_loss 0.43409
wandb: 
wandb: 🚀 View run exp007_fold1 at: https://wandb.ai/welshonionman/SenNet/runs/66oai3s4
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: /kaggle/wandb/run-20240102_001914-66oai3s4/logs
